In [1]:
import gradio as gr
import pandas as pd
from gtts import gTTS
import os

# User data (PIN, name, balance)
users = {
    123456: {'name': 'User1', 'balance': 10000},
    654321: {'name': 'User2', 'balance': 15000}
}

current_user = None

# File path for storing transaction history
def get_transaction_file():
    """Return the path to the transaction CSV file for the current user."""
    return f"{current_user['name']}_transactions.csv"

def load_transactions():
    """Load the transaction history from the CSV file."""
    file_path = get_transaction_file()
    if os.path.exists(file_path):
        return pd.read_csv(file_path)
    else:
        return pd.DataFrame(columns=['Transaction Type', 'Amount', 'Balance'])

def save_transactions(transactions_df):
    """Save the transaction history to the CSV file."""
    file_path = get_transaction_file()
    transactions_df.to_csv(file_path, index=False)

def print_passbook():
    """Display the user's passbook in a DataFrame format and return an audio narration."""
    transactions_df = load_transactions()
    
    if transactions_df.empty:
        text = "No transactions yet."
        return transactions_df, create_audio(text)
    
    # Generate the text for speech
    text = "Here is your passbook: \n"
    for _, row in transactions_df.iterrows():
        text += f"{row['Transaction Type']} of {row['Amount']}. Balance after transaction: {row['Balance']}.\n"
    
    # Create the audio from the text
    audio_path = create_audio(text)
    return transactions_df, audio_path

def create_audio(text):
    """Convert text to speech using gTTS and return the audio file path."""
    tts = gTTS(text)
    audio_path = "passbook_audio.mp3"
    tts.save(audio_path)
    return audio_path

def op(choice, amt=0):
    """Main operation for handling user transactions."""
    global current_user
    balance = current_user['balance']
    transactions_df = load_transactions()
    
    if choice == "Deposit":
        balance += amt
        transactions_df = transactions_df.append({'Transaction Type': 'Deposit', 'Amount': amt, 'Balance': balance}, ignore_index=True)
        save_transactions(transactions_df)
        return f"Deposit successful! New balance: {balance}", transactions_df, None
    elif choice == "Withdraw":
        if amt <= balance:
            balance -= amt
            transactions_df = transactions_df.append({'Transaction Type': 'Withdraw', 'Amount': amt, 'Balance': balance}, ignore_index=True)
            save_transactions(transactions_df)
            return f"Withdrawal successful! New balance: {balance}", transactions_df, None
        else:
            return "Insufficient balance!", transactions_df, None
    elif choice == "Balance Enquiry":
        return f"Your current balance is: {balance}", transactions_df, None
    elif choice == "Passbook":
        # Return the DataFrame of transactions and audio narration
        passbook_df, audio_path = print_passbook()
        return "Here is your passbook:", passbook_df, audio_path
    elif choice == "Exit":
        return "Thank you! Exiting the system.", transactions_df, None
    else:
        return "Invalid choice.", transactions_df, None

def login(pin):
    """Login function to handle user authentication."""
    global current_user
    if pin in users:
        current_user = users[pin]
        return f"Logged in successfully as {current_user['name']}!"
    else:
        return "Incorrect PIN. Please try again."

def atm_interface(pin, choice, amt):
    """Handle the entire ATM interface logic."""
    login_message = login(pin)
    if "successfully" in login_message:
        result_text, result_df, audio_path = op(choice, amt)
        return login_message + "\n" + result_text, result_df, audio_path
    else:
        return login_message, pd.DataFrame(), None

# Set up Gradio inputs and outputs
pin_input = gr.Number(label="Enter PIN")
choice_input = gr.Radio(["Deposit", "Withdraw", "Balance Enquiry", "Passbook", "Exit"], label="Choose an Operation")
amount_input = gr.Number(label="Enter Amount", value=0)

output_text = gr.Textbox()
output_dataframe = gr.Dataframe()
output_audio = gr.Audio()

# Create the Gradio interface
gr.Interface(fn=atm_interface, 
             inputs=[pin_input, choice_input, amount_input], 
             outputs=[output_text, output_dataframe, output_audio]).launch()





Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


C:\Users\91990\AppData\Local\Temp\ipykernel_21820\3894955948.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transactions_df = transactions_df.append({'Transaction Type': 'Withdraw', 'Amount': amt, 'Balance': balance}, ignore_index=True)
